In [1]:
# Dependencies
# import numpy as np
import pandas as pd
import datetime as dt
import requests
import json

# Extract: Data Pull

## Extract Data & Clean

### API Tracking data from CDC

#### Max temp days

In [2]:
# URL for GET requests to retrieve data
version = "v1"
measureId = 423 # Requires a valid measureID
stratificationLevelId = 82 # Requires a valid stratificationLevelId.
geographicTypeIdFilter = 1 # Requires a valid geographicTypeID or ALL.
geographicItemsFilter = 4 # Filter to retrieve only certain geographicItems.
temporalTypeIdFilter = 1 # Requires a valid temporal type id matching temporal items (e.g., 1)
temporalItemsFilter = "2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011" # Requires valid temporal items entries (e.g., 2020) separated by comma.
isSmoothed = 0 # 0 not smoothed- most data isn't
getFullCoreHolder = 0 # Requires either a one or zero (true or false) for fetching the full core holder. 


# Build the endpoint URL
MaxTempDays_url = f"https://ephtracking.cdc.gov/apigateway/api/{version}/getCoreHolder/{measureId}/{stratificationLevelId}/{geographicTypeIdFilter}/{geographicItemsFilter}/{temporalTypeIdFilter}/{temporalItemsFilter}/{isSmoothed}/{getFullCoreHolder}?TemperatureHeatIndexId=1&RelativeThresholdId=1" #{getFullCoreHolder}

response = requests.get(MaxTempDays_url).json()

# Isolate tableResult for easy reading
data = response["tableResult"]

#for index, element in enumerate(data):
#  print(index, ":", element)

In [18]:
# Create an empty list to hold summaries
summary_list = []

# loop through the data
for bob in data:
    county = bob["geo"]
    countyID= bob["geoId"]
    year = bob["temporal"]
    days = bob["dataValue"]
    
    # create summary dictionary
    summary_dict = {
        "county": county,
        "countyID" : countyID,
        "year": year,
        "ext_heat_days" : days
    }
    
    # Append the sumary dict to the list
    summary_list.append(summary_dict)

EHD = pd.DataFrame(summary_list) 
EHD

,county,countyID,year,ext_heat_days
0,Apache,04001,2011,19
1,Apache,04001,2012,19
2,Apache,04001,2013,18
3,Apache,04001,2014,5
4,Apache,04001,2015,18
...,...,...,...,...
160,Yuma,04027,2017,21
161,Yuma,04027,2018,19
162,Yuma,04027,2019,22
163,Yuma,04027,2020,36


In [19]:
EHD = EHD.reset_index()
EHD.head()

,index,county,countyID,year,ext_heat_days
0,0,Apache,04001,2011,19
1,1,Apache,04001,2012,19
2,2,Apache,04001,2013,18
3,3,Apache,04001,2014,5
4,4,Apache,04001,2015,18


In [20]:
EHD.rename(columns={'index':'key'}, inplace=True)
EHD.head()

,key,county,countyID,year,ext_heat_days
0,0,Apache,04001,2011,19
1,1,Apache,04001,2012,19
2,2,Apache,04001,2013,18
3,3,Apache,04001,2014,5
4,4,Apache,04001,2015,18


In [6]:
# Export the cleaned DataFrame as a CSV file. 
EHD.to_csv("data/ExtremeHeatDays.csv", encoding='utf8', index=False)

## CSV to JSON files

In [12]:
import csv
import json
 
 
# Function to convert a CSV to JSON
# Takes the file paths as arguments
def make_json(csv_file_path, jsonFilePath):
     
    # create a dictionary
    data = {}
     
    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        # Convert each row into a dictionary
        # and add it to data
        for rows in csvReader:
             
            # Assuming a column named 'No' to
            # be the primary key
            key = rows['key']
            data[key] = rows
 
    # Open a json writer, and use the json.dumps()
    # function to dump data
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))
         

In [13]:
# Driver Code
 
# Decide the two file paths according to your
# computer system
csvFilePath = r'data/merge_df.csv'
jsonFilePath = r'data/HRI.json'
 
# Call the make_json function
make_json(csvFilePath, jsonFilePath)

# Load: SQLite database

In [21]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func, distinct

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, Boolean

from pathlib import Path

In [39]:
Path('data/hri.sqlite').touch()

In [40]:
# create engine to HRI.sqlite
engine = create_engine("sqlite:///data/hri.sqlite")

In [41]:
engine.execute('''CREATE TABLE extreme_heat_days (key int PRIMARY KEY, county text, countyID int, year text, ext_heat_days float)''')

In [42]:
# load the data into a Pandas DataFrame and write to sqlite table
extreme_heat_days = pd.read_csv('data/ExtremeHeatDays.csv')
extreme_heat_days.to_sql('extreme_heat_days', engine, if_exists='append', index = False)

165

In [43]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(autoload_with=engine)
Base.classes.keys() #['ed_visit_35', 'extreme_heat_days']

['extreme_heat_days']

# Building Queries from SQLite database
These queries will them be used in the flask but need to check to see if they work

In [44]:
# Create our session (link) from Python to the DB
session = Session(bind=engine)

inspector = inspect(engine)

In [45]:
columns = inspector.get_columns('extreme_heat_days')
for column in columns:
    print(column["name"], column["type"])

key INTEGER
county TEXT
countyID INTEGER
year TEXT
ext_heat_days FLOAT


In [50]:
session.query(extreme_heat_days.year).first()

ArgumentError: Column expression or FROM clause expected, got 0      2011
1      2012
2      2013
3      2014
4      2015
       ... 
160    2017
161    2018
162    2019
163    2020
164    2021
Name: year, Length: 165, dtype: int64.

In [38]:
### Using that date, get the previous 5 years of data 
##### by querying the previous 5 y of data.

prev5_ehd = session.query(extreme_heat_days.year, extreme_heat_days.ext_heat_days ).\
    filter(extreme_heat_days.year <= '2021').\
    filter(extreme_heat_days.year > '2017').\
    order_by(extreme_heat_days.year).all()

pprint(prev5_ehd)

ArgumentError: Column expression or FROM clause expected, got 0      2011
1      2012
2      2013
3      2014
4      2015
       ... 
160    2017
161    2018
162    2019
163    2020
164    2021
Name: year, Length: 165, dtype: int64.